### Download the coops predictions

In [1]:
import pandas as pd
import requests
from io import StringIO
from pathlib import Path

# ============================================================
# SETTINGS
# ============================================================
STATION = "8654467"
OUTDIR = Path("coops_8654467")
OUTDIR.mkdir(exist_ok=True)

OUTCSV = OUTDIR / "COOPS_8654467_predictions_6min_MSL_metric_GMT.csv"

START = pd.Timestamp("2010-04-01")   # station start
END   = pd.Timestamp.today().normalize()

BASE = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

# ============================================================
# FETCH ONE MONTH
# ============================================================
def fetch_predictions(beg, end):
    params = dict(
        product="predictions",
        application="khse_setup_calibration",
        begin_date=beg.strftime("%Y%m%d"),
        end_date=end.strftime("%Y%m%d"),
        station=STATION,
        datum="MSL",
        units="metric",
        time_zone="gmt",
        interval="6",
        format="csv",
    )
    r = requests.get(BASE, params=params, timeout=60)
    r.raise_for_status()
    txt = r.text.strip()
    if not txt or txt.lower().startswith("error"):
        return None
    df = pd.read_csv(StringIO(txt))
    df.columns = [c.strip() for c in df.columns]
    return df

# ============================================================
# DOWNLOAD ALL MONTHS
# ============================================================
chunks = []

for beg in pd.date_range(START, END, freq="MS"):
    fin = beg + pd.offsets.MonthEnd(0)
    if fin > END:
        fin = END

    df = fetch_predictions(beg, fin)
    if df is not None:
        chunks.append(df)

if not chunks:
    raise RuntimeError("No prediction data returned from CO-OPS.")

pred = pd.concat(chunks, ignore_index=True)

# Parse time column
pred["Date Time"] = pd.to_datetime(pred["Date Time"], errors="coerce")
pred = pred.dropna(subset=["Date Time"]).sort_values("Date Time")

# Save
pred.to_csv(OUTCSV, index=False)

print(f"Wrote {len(pred):,} rows to:")
print(OUTCSV.resolve())

# ============================================================
# QUICK SANITY CHECK
# ============================================================
print(pred.head())
print(pred.tail())


Wrote 1,379,040 rows to:
F:\crs\src\Dorian_recovery\coops_8654467\COOPS_8654467_predictions_6min_MSL_metric_GMT.csv
            Date Time  Prediction
0 2010-04-01 00:00:00       0.084
1 2010-04-01 00:06:00       0.088
2 2010-04-01 00:12:00       0.091
3 2010-04-01 00:18:00       0.094
4 2010-04-01 00:24:00       0.097
                  Date Time  Prediction
1379035 2025-12-23 23:30:00      -0.107
1379036 2025-12-23 23:36:00      -0.105
1379037 2025-12-23 23:42:00      -0.104
1379038 2025-12-23 23:48:00      -0.103
1379039 2025-12-23 23:54:00      -0.101
